# 00: Basics

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It explains how to use an interactive tool employed by subsequent notebooks for visualising and animating algorithms.
Furthermore, it introduces a few important basics that are frequently needed in algorithm implementations.

## Table of Contents

1. Introduction and Setup  
2. Visualisation  
3. Geometry  
    3.1. Robustness  
    3.2. Orientation Test  
    3.3. Intersection Computation for Two Line Segments  
4. Data Structures  
    4.1. Balanced Binary Tree  
    4.2. Doubly-Connected Edge List for a Simple Polygon  
5. References  

## 1. Introduction and Setup

This notebook reimplements some of the elements living in modules to be reused in subsequent notebooks:

* <tt>modules.visualisation:</tt> Module for visualisation purposes.

* <tt>modules.geometry:</tt> Module for geometric primitives and operations.

* <tt>modules.data_structures:</tt> Module for generic data structures.

We now import everything we'll need throughout this notebook from the [Python Standard Library](https://docs.python.org/3/library/).

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from enum import auto, Enum
from itertools import count
import math
from typing import Any, Generic, Optional, SupportsFloat, TypeVar, Union

## 2. Visualisation

The main part of our <tt>visualisation</tt> module is the <tt>VisualisationTool</tt> class.
See below for an example image of the tool as it appears in [notebook no. 02](./02-LineSegmentIntersection.ipynb) about the line segment intersection problem.
The input (here, a set of line segments) is coloured <font color='orange'>orange</font> and the algorithm output (here, a set of intersection points) is marked in <font color='blue'>blue</font>.

Furthermore, the <tt>visualisation</tt> module exports some other elements that support the tool.
These elements include instance handles (e.g. <tt>LineSegmentSetInstance</tt>) specifying input types and interactions as well as drawing modes (e.g. <tt>PointsMode</tt>) specifying the mechanics of drawings and animations.
Drawing modes have customisation parameters, such as the radius of drawn points/vertices.
All subsequent notebooks employ appropriate instance handles and drawing modes with reasonable defaults.

<img style='float: left;' src='./images/00-image00.png'>

Invoking the tool is done by calling the <tt>display()</tt> method of a <tt>VisualisationTool</tt> object.
Cells with this method call are already included in subsequent notebooks.
Before executing such a cell, a static image is displayed in place of the tool.
These images serve as animation previews by depicting an algorithm step on a random input instance.
In order to successfully invoke the tool, all prior notebook cells should be executed beforehand to make sure everything works.
Obviously, you also need a suitable environment including a Python kernel.
If you don't know how to access the notebooks within an execution environment, you can consult the [README](/README.md) for different access possibilities.

The visualisation tool contains an interactive canvas on its left side.
Width and height of the canvas are choosable during tool initialisation.
Input instances can be constructed by clicking on the canvas.
This adds a point to the current instance, up to a maximum of 999 points.
The instance size (usually given the name $n$ when analysing algorithm running time) is shown beneath the canvas.
It can differ from the number of added points, which is then indicated separately as seen in the image.
Here, that distinction applies since every line segment comprises two points.
On the right side of the tool are control widgets divided into multiple sections, which will be described next.

* **Canvas:**

  * The *Clear* button clears the current instance, removing it from the canvas.

  * The *Random* button generates a new random instance, replacing the current instance on the canvas.
    During generation a message saying "GENERATING" is displayed next to the *Points* option.

* **Options:**

  * The *Animations* checkbox toggles whether to animate algorithms instead of directly drawing algorithm outputs onto the canvas.

  * The *Speed* option is only visible if the *Animations* checkbox is checked.
    It controls the animation speed on a scale from 1 to 10, where an exponential function is applied to the chosen value.
    Note that it's not possible to pause or stop a running animation, so animations on large instances should best be started with a high speed value.
    (If you're stuck in a long animation or something unforeseen happens, you can still always restart the execution kernel.)

  * The *Points* option controls how many points the random instances generated by the *Random* button consist of.

* **Examples:** Each example instance registered with the <tt>VisualisationTool.register_example_instance(...)</tt> method receives its own button.
  Clicking on such a button loads the corresponding instance, replacing the current instance on the canvas.

* **Algorithms:** Each algorithm registered with the <tt>VisualisationTool.register_algorithm(...)</tt> method receives its own button.
  Clicking on such a button executes the corresponding algorithm on the current instance, optionally animates algorithm steps and finally draws the algorithm output onto the canvas.
  The algorithm output is removed from the canvas whenever another algorithm execution starts or the input instance changes.

* **Messages:** After clicking on an algorithm button, a message regarding the execution of the algorithm is displayed next to the button.
  During execution it says "RUNNING", while during animation it says "<font color='blue'>ANIMATING</font>".
  After that, the running time of the algorithm execution (not including animation) is displayed until the algorithm is executed again or the input instance changes.
  Should the execution fail for some reason, then the message says "<font color='red'>ERROR</font>" instead and hovering over it yields an error description.

## 3. Geometry

The <tt>geometry</tt> module contains the primitives <tt>Point</tt> and <tt>LineSegment</tt> for 2D geometry, which subsequent notebooks are based on.
It also exports classes acting as interfaces for the visualisation tool, namely <tt>PointSequence</tt> (essentially a [list](https://docs.python.org/3/library/stdtypes.html#lists) of points) and <tt>PointSequenceDict</tt> (essentially an [OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict) with points as keys and arbitrary values).
Those classes support only basic operations but implicitly store animation events for the tool.
Every registered algorithm thus needs to return such a class object.

In this section, we'll reimplement operations for the geometric primitives that are provided by the <tt>geometry</tt> module:
The *orientation test* for a point with respect to a directed line segment, and the *intersection computation* for two line segments.
These operations are fundamental in computational geometry and used frequently in subsequent notebooks, especially the orientation test.
Hence, the notion of *robustness* is very relevant for their implementations, so let's take a quick look at that first.

### 3.1. Robustness

Implementations of geometric algorithms might return wrong outputs for specific inputs.
This can happen for **degenerate cases**, which are annoying edge cases often ignored in algorithm descriptions (see \[1, pp. 8–9\]).
But even if all cases are handled correctly, errors are still able to arise because [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic) isn't exact.
Some algorithms particularly suffer from so-called **robustness issues**, where small rounding errors can lead to big deviations in the output (see \[1, p. 5 and pp. 9–10\]).

The easiest solution for these issues is to perform comparisons using an absolute epsilon value, i.e. two floating-point numbers are considered equal as long as their difference isn't greater than the chosen epsilon value.
We use this approach throughout our modules and notebooks.

In [2]:
EPSILON: float = 1e-9    # Chosen by testing currently implemented algorithms with the visualisation tool.

However, this isn't a good solution since the spacing between two adjacent representable floating-point numbers grows with their magnitude (see \[2\]).
That spacing is called the *ULP* of a float, short for [unit in the last place](https://en.wikipedia.org/wiki/Unit_in_the_last_place).
As an example, let's look at the ULPs of powers of 10 and what happens when they become larger than the absolute epsilon value.

In [3]:
for n in map(lambda i: 10.0 ** i, count()):
    print(f"Float {n:.1e} has ULP {math.ulp(n):.16e}")
    assert n + EPSILON > n, "Floats doing float things."

Float 1.0e+00 has ULP 2.2204460492503131e-16
Float 1.0e+01 has ULP 1.7763568394002505e-15
Float 1.0e+02 has ULP 1.4210854715202004e-14
Float 1.0e+03 has ULP 1.1368683772161603e-13
Float 1.0e+04 has ULP 1.8189894035458565e-12
Float 1.0e+05 has ULP 1.4551915228366852e-11
Float 1.0e+06 has ULP 1.1641532182693481e-10
Float 1.0e+07 has ULP 1.8626451492309570e-09
Float 1.0e+08 has ULP 1.4901161193847656e-08


AssertionError: Floats doing float things.

In consequence, absolute epsilon comparisons only work for relatively small numbers and are inadequate for general purpose implementations.
They're mostly fine for our use case due to the visualisation tool's coordinate space being bounded by the canvas width and height, which should be chosen such that they fit on a screen.
When comparing larger numbers, comparisons based on relative epsilon values or ULPs could be used instead.
An approachable introduction to comparing floats is given by \[2\].

Alternatively, one can use exact arithmetic.
This is achievable by explicitly storing numerators and denominators of rational numbers, or through arbitrary precision arithmetic as provided by [Python's decimal library](https://docs.python.org/3/library/decimal.html).
Unfortunately, both mentioned options are usually quite a bit slower than fixed precision floating-point arithmetic on typical hardware.
That's why there's been research on more advanced methods that aim to combine exactness with a moderate performance overhead (see \[1, p. 14\]).
For instance, techniques for quickly computing geometric predicates (such as the orientation discussed next) using adaptive precision arithmetic are presented in \[3\].

### 3.2. Orientation Test

The 2D orientation test determines the location of a point $p \in \mathbb{R}^2$ with respect to a directed line segment starting at a source $s \in \mathbb{R}^2$ and ending at a target $t \in \mathbb{R}^2$ with $s \neq t$.
As illustrated by the image below (created using [Ipe](https://ipe.otfried.org/)), there are five possible configurations from the viewpoint of an imaginary agent standing at $s$ and looking towards $t$:
The point $p$ can lie to the agent's *left*, or to their *right*, or on the line segment *between* $s$ and $t$ (both inclusive), or on the corresponding line *before $s$*, or on the line *behind $t$*.

<img style='float: left;' src='./images/00-image01.png'>

We define an [Enum](https://docs.python.org/3/library/enum.html) called <tt>Orientation</tt> (or <tt>ORT</tt> for short) encoding these possibilities:

In [4]:
class Orientation(Enum):
    LEFT = auto()
    RIGHT = auto()
    BETWEEN = auto()
    BEFORE_SOURCE = auto()
    BEHIND_TARGET = auto()

ORT = Orientation

The orientation can be computed using the sign of a determinant (see \[4, pp. 10–11\]).
To make this a bit more intuitive, said determinant conincides with the *perp dot product* of the vectors $t - s$ and $p - s$.
The well-known [dot product](https://en.wikipedia.org/wiki/Dot_product) of two 2D vectors $v = (v_x, v_y) \in \mathbb{R}^2$ and $w = (w_x, w_y) \in \mathbb{R}^2$ is defined as $v \cdot w := v_x \cdot w_x + v_y \cdot w_y$ and satisfies $v \cdot w = \|v\| \cdot \|w\| \cdot \cos(\theta)$, where $\theta$ is the (smaller) angle between $v$ and $w$.
The perp dot product of $v$ and $w$ corresponds to $v^\perp \cdot w$, where $v^\perp := (-v_y, v_x)$ is perpendicular to $v$.
It satisfies $|v^\perp \cdot w| = \|v\| \cdot \|w\| \cdot \sin(\theta)$ and can thus be interpreted as the signed area of the paralleogram induced by $v$ and $w$ (see \[5, pp. 138–140\]).

Intuitively, $p$ lies on the line through $s$ and $t$ if and only if the area of the paralleogram induced by $t - s$ and $p - s$ vanishes, i.e. $(t - s)^\perp \cdot (p - s) = 0$.
The location of $p$ on the line is then parametrised by the equation $p = s + a \cdot (t - s)$ for some scalar $a \in \mathbb{R}$.
Here, $a < 0$ means that $p$ lies before $s$, and $a \in [0, 1]$ means that $p$ lies between $s$ and $t$, while $a > 1$ means that $p$ lies behind $t$.
We can solve the equation for $a$ with the help of dot products, yielding $a = (p - s) \cdot (t - s) / (t - s) \cdot (t - s)$.

Otherwise, the sign of the perp dot product $(t - s)^\perp \cdot (p - s)$ immediately tells us the orientation.
If it's positive, then $p$ is to the left, and if it's negative, then $p$ is to the right (see \[4, pp. 10–11\] and \[5, pp. 139\]).
The upcoming code uses the alternative perp dot product $(p - s)^\perp \cdot (t - s) = - (t - s)^\perp \cdot (p - s)$ instead, which switches the sign due to antisymmetry.
It also employs a choosable absolute epsilon value for checking whether the perp dot product is $0$ (see the previous subsection on robustness).

Furthermore, note that the <tt>geometry</tt> module doesn't distuinguish between points and vectors as well as that this reimplementation doesn't include all methods of the <tt>Point</tt> class from the module.
Besides the discussed operations, it just contains properties and magic methods that are used by the intersection computation.

In [5]:
class Point:
    def __init__(self, x: SupportsFloat, y: SupportsFloat):
        if not math.isfinite(x) or not math.isfinite(y):
            raise ValueError("The x and y values of a point must be finite numbers.")

        self._x = float(x)
        self._y = float(y)

    ## Operations

    def orientation(self, source: Point, target: Point, epsilon: float = EPSILON) -> Orientation:
        if source == target:
            raise ValueError("Source and target need to be two different points.")

        self_direction = self - source
        target_direction = target - source
        signed_area = self_direction.perp_dot(target_direction)

        if signed_area < -epsilon:
            return ORT.LEFT
        elif signed_area > epsilon:
            return ORT.RIGHT
        else:
            a = self_direction.dot(target_direction) / target_direction.dot(target_direction)
            # We don't need epsilon here, because the calculation of `a` ensures that
            # `a == 0.0` if `self == source`, whereas `a == 1.0` if `self == target`.
            if a < 0.0:
                return ORT.BEFORE_SOURCE
            elif a > 1.0:
                return ORT.BEHIND_TARGET
            else:
                return ORT.BETWEEN

    def dot(self, other: Point) -> float:
        return self._x * other._x + self._y * other._y

    def perp_dot(self, other: Point) -> float:
        return self._x * other._y - self._y * other._x

    ## Properties

    @property
    def x(self) -> float:
        return self._x

    @property
    def y(self) -> float:
        return self._y

    ## Magic methods

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, Point):
            return NotImplemented

        return self.x == other.x and self.y == other.y

    def __add__(self, other: Any) -> Point:
        if not isinstance(other, Point):
            return NotImplemented

        return Point(self._x + other._x, self._y + other._y)

    def __sub__(self, other: Any) -> Point:
        if not isinstance(other, Point):
            return NotImplemented

        return Point(self._x - other._x, self._y - other._y)

    def __rmul__(self, other: Any) -> Point:
        try:
            x = float(other * self._x)
            y = float(other * self._y)
        except Exception:
            return NotImplemented

        return Point(x, y)

### 3.3. Intersection Computation for Two Line Segments

Given two 2D line segments $S_1$ and $S_2$, the goal is to compute their intersection $S_1 \cap S_2$.
We write $u_i \in \mathbb{R}^2$ for the upper endpoint and $l_i \in \mathbb{R}^2$ for the lower endpoint of $S_i$, where $u_i \neq l_i$ and $i \in \{1,2\}$.
(The distinction between upper and lower endpoints doesn't actually matter here, but it'll be useful in subsequent notebooks.)
Just like for the orientation test, there's a fairly straightforward algorithm making use of perp dot products (see the previous subsection for a short introduction to them).
The algorithm again employs a choosable absolute epsilon value for checking perp dot products and related (see the subsection on robustness).

First of all, we check whether $(u_1 - l_1)^\perp \cdot (u_2 - l_2) \neq 0$ holds.
If so, the line segments $S_1$ and $S_2$ aren't parallel, meaning that the corresponding lines intersect in exactly one point.
That point is parametrised by the equation $l_1 + a \cdot (u_1 - l_1) = l_2 + b \cdot (u_2 - l_2)$ for some scalars $a, b \in \mathbb{R}$.
We can solve the equation for $a$ and $b$ with the help of perp dot products (see \[5, pp. 141–142\]).
Then the intersection point of the corresponding lines is contained in $S_1 \cap S_2$ if both $a \in [0, 1]$ and $b \in [0, 1]$ apply, which is illustrated by case A1 in the following image (again created using [Ipe](https://ipe.otfried.org/)).
Otherwise, the given line segments don't intersect, even though the corresponding lines do (see case A2).

<img style='float: left;' src='./images/00-image02.png'>

If $(u_1 - l_1)^\perp \cdot (u_2 - l_2) = 0$ holds instead, the line segments $S_1$ and $S_2$ are parallel.
Hence, they can't intersect (see case B) unless they lie on the same corresponding line.
The latter is equivalent to $(l_2 - l_1)^\perp \cdot (u_2 - l_2) = 0$ and allows us to parametrise the locations of one segment's endpoints on the line using the endpoints of the other segment.
This can be done with the help of dot products, similar to the previous subsection, and yields two scalars $a_{\text{lower}}, a_{\text{upper}} \in \mathbb{R}$.
Finally, we clip $a_{\text{lower}}$ to the interval $[0, \infty)$ and $a_{\text{upper}}$ to the interval $(-\infty, 1]$.
Those clipped values allow us to distinguish the following cases for the intersection of parallel line segments $S_1$ and $S_2$:

* The segments share an endpoint (see case C1, where $l_1 = u_2$).
  Note that this can also happen in case A1, but doesn't need to be treated separately there.
  
* The segments overlap in a line segment (see case C2).
  Note that one of the line segments could also be completely contained in the other one.

* The segments don't intersect at all despite sharing a corresponding line (see case C3).

In [6]:
class LineSegment:
    def __init__(self, p: Point, q: Point):
        if p == q:
            raise ValueError("A line segment needs two different endpoints.")
        elif p.y > q.y or (p.y == q.y and p.x < q.x):
            self._upper = p
            self._lower = q
        else:
            self._upper = q
            self._lower = p

    ## Operation(s)

    def intersection(self, other: LineSegment, epsilon: float = EPSILON) -> Union[Point, LineSegment, None]:
        self_direction = self._upper - self._lower
        other_direction = other._upper - other._lower
        lower_offset = other._lower - self._lower
        signed_area_sd_od = self_direction.perp_dot(other_direction)
        signed_area_lo_od = lower_offset.perp_dot(other_direction)
        signed_area_lo_sd = lower_offset.perp_dot(self_direction)

        if abs(signed_area_sd_od) > epsilon:
            a = signed_area_lo_od / signed_area_sd_od
            b = signed_area_lo_sd / signed_area_sd_od
            if -epsilon <= a <= 1.0 + epsilon and -epsilon <= b <= 1.0 + epsilon:
                return self._lower + a * self_direction
            else:
                return None

        # Check both signed areas to ensure consistency and increase robustness.
        if abs(signed_area_lo_od) <= epsilon or abs(signed_area_lo_sd) <= epsilon:
            self_direction_dot = self_direction.dot(self_direction)
            upper_offset = other._upper - self._lower
            a_lower = lower_offset.dot(self_direction) / self_direction_dot
            a_upper = upper_offset.dot(self_direction) / self_direction_dot

            # The inner min/max operations aren't needed in theory, because `a_lower < a_upper`
            # should always hold. However, inaccuracies might somehow invalidate that.
            a_lower_clipped = max(0.0, min(a_lower, a_upper))
            a_upper_clipped = min(1.0, max(a_lower, a_upper))
            upper = self._lower + a_upper_clipped * self_direction
            if a_lower_clipped == a_upper_clipped:
                return upper
            elif a_lower_clipped < a_upper_clipped:
                lower = self._lower + a_lower_clipped * self_direction
                return LineSegment(upper, lower)
            else:
                return None

        return None

    ## Properties
    
    @property
    def upper(self) -> Point:
        return self._upper

    @property
    def lower(self) -> Point:
        return self._lower

## 4. Data Structures

To wrap the basics up, let's take a look at the data structures provided by the <tt>data_structures</tt> module.

### 4.1 Balanced Binary Tree

The notion of a [balanced binary tree](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) should be familiar to the reader.
In such a tree, the searchable position of a node is determined by the *key* it stores.
This requires a strict total order on the set of all possible keys.
To allow for user-defined orders, our implementation makes use of a generic [abstract base class](https://docs.python.org/3/library/abc.html) called <tt>Comparator</tt>.
Inheriting classes need a <tt>compare(item, key)</tt> method comparing an *item* (e.g. a new key to be inserted) to an existing node key in the tree.
The return value of type <tt>ComparisonResult</tt> (or <tt>CR</tt> for short) specifies whether the item comes before, matches with or comes after the key.
Note that items don't always have to be valid keys depending on the use case.

In [7]:
class ComparisonResult(Enum):
    BEFORE = -1
    MATCH = 0
    AFTER = 1

CR = ComparisonResult
K = TypeVar("K")
V = TypeVar("V")

class Comparator(ABC, Generic[K]):
    @abstractmethod
    def compare(self, item: Any, key: K) -> ComparisonResult:
        pass

The balanced binary tree from the <tt>data_structures</tt> module is an [AA tree](https://en.wikipedia.org/wiki/AA_tree), originally proposed in \[6\].
It has two balancing operations called *skew* and *split*, which are used after node insertion and deletion to restructure nodes in the tree according to their *levels* (see \[6, p. 62\]).
The code for the skew, split, insert and delete operations is so compact that it fits on a single page (see \[6, p. 66\]).
Our implementation in the <tt>data_structures</tt> module supports even more operations, but here we'll only reimplement the insert operation and all that's needed for it, including skew and split.
Note that our code for those balancing operations differs from the one in \[6\] in order to avoid allocating a new <tt>Node</tt> object, but it yields the same result.
We also store a *value* in each node, making it possible to use the binary tree as a dictionary by looking up values of inserted keys.

In [8]:
class Node(Generic[K, V]):
    def __init__(self):
        self._key: Optional[K] = None
        self._value: Optional[V] = None
        self._level: int = 0
        self._left: Optional[Node[K, V]] = None
        self._right: Optional[Node[K, V]] = None

    ## Operations

    def insert(self, key: K, value: V, comparator: Comparator[K]) -> bool:
        if self.is_empty():
            self._make_leaf(key, value)
            return False

        cr = comparator.compare(key, self._key)
        if cr is CR.BEFORE:
            was_key_present = self._left.insert(key, value, comparator)
        elif cr is CR.AFTER:
            was_key_present = self._right.insert(key, value, comparator)
        else:
            was_key_present = True

        if not was_key_present:
            self._skew()
            self._split()

        return was_key_present

    def is_empty(self) -> bool:
        return self._level == 0

    ## Helper operations

    def _make_leaf(self, key: K, value: V):
        self._key = key
        self._value = value
        self._level = 1
        self._left = Node()
        self._right = Node()

    def _skew(self):
        if not self.is_empty() and self._left._level == self._level:
            self._key, self._left._key = self._left._key, self._key
            self._value, self._left._value = self._left._value, self._value

            self._left, self._right = self._right, self._left
            self._left, self._right._left = self._right._left, self._left
            self._right._left, self._right._right = self._right._right, self._right._left

    def _split(self):
        if not self.is_empty() and not self._right.is_empty() and self._right._right._level == self._level:
            self._key, self._right._key = self._right._key, self._key
            self._value, self._right._value = self._right._value, self._value

            self._left, self._right = self._right, self._left
            self._right, self._left._right = self._left._right, self._right
            self._left._left, self._left._right = self._left._right, self._left._left

            self._level += 1

The <tt>data_structures</tt> module exports the wrapper classes <tt>BinaryTree</tt> (not supporting values) and <tt>BinaryTreeDict</tt> (supporting values) instead of the <tt>Node</tt> class itself.

In [9]:
class BinaryTree(Generic[K]):
    def __init__(self, comparator: Comparator[K]):
        self._root: Node[K, None] = Node()
        self._comparator = comparator
    
    def insert(self, key: K) -> bool:
        return self._root.insert(key, None, self._comparator)

    def is_empty(self) -> bool:
        return self._root.is_empty()

class BinaryTreeDict(Generic[K, V]):
    def __init__(self, comparator: Comparator[K]):
        self._root: Node[K, V] = Node()
        self._comparator = comparator

    def insert(self, key: K, value: V) -> bool:
        return self._root.insert(key, value, self._comparator)

    def is_empty(self) -> bool:
        return self._root.is_empty()

### 4.2. Doubly-Connected Edge List for a Simple Polygon

Another data structure that's very useful for geometric algorithms is the *doubly-connected edge list*, or *DCEL* for short.
As its name implies, the DCEL is centered around edges.
Each edge connects two vertices and is split into two *half-edges* pointing into opposite directions, such that every half-edge is incident to exactly one face and is oriented with that face to its left.
This enables anticlockwise traversals of bounded faces.
More information on DCELs is given in the lectures and in \[1, section 2.2\].

The <tt>data_structures</tt> module currently doesn't contain a general DCEL, but one for modelling a [simple polygon](https://en.wikipedia.org/wiki/Simple_polygon).
Objects of class <tt>DoublyConnectedSimplePolygon</tt> allow adding vertices to a polygonal path, except when doing so would lead to irreversible intersections in the path.
They also keep track of which half-edges are going in anticlockwise direction, and manage the provisional closing edge between the two vertices added first and last.
That's why the operations of <tt>DoublyConnectedSimplePolygon</tt> are already pretty intricate.
We won't reimplement them here.

Instead we'll showcase the underlying <tt>Vertex</tt> and <tt>HalfEdge</tt> classes, which are also exported by the module.
Each vertex stores a point and the half-edge that's going out from it in anticlockwise direction.
Meanwhile, each half-edge stores its origin vertex and its twin half-edge as well as the previous and next half-edge in the closed polygonal path (or the reverse of that path, depending on the half-edge's direction).
The classes have additional properties for quick access, such as the destination vertex of a half-edge.

In [10]:
class Vertex:
    def __init__(self, point: Point):
        self._point = point
        self._edge: HalfEdge = HalfEdge(self)

    ## Properties

    @property
    def point(self) -> Point:
        return self._point

    @property
    def x(self) -> float:
        return self._point.x

    @property
    def y(self) -> float:
        return self._point.y

    @property
    def edge(self) -> HalfEdge:
        return self._edge

class HalfEdge:
    def __init__(self, origin: Vertex):
        self._origin = origin
        self._twin: HalfEdge = self
        self._prev: HalfEdge = self
        self._next: HalfEdge = self

    ## Properties

    @property
    def origin(self) -> Vertex:
        return self._origin

    @property
    def destination(self) -> Vertex:
        return self._twin._origin

    @property
    def upper_and_lower(self) -> tuple[Vertex, Vertex]:
        p, q = self._origin, self.destination
        if p.y > q.y or (p.y == q.y and p.x < q.x):
            return p, q
        else:
            return q, p

    @property
    def twin(self) -> HalfEdge:
        return self._twin

    @property
    def prev(self) -> HalfEdge:
        return self._prev

    @property
    def next(self) -> HalfEdge:
        return self._next

## 5. References

\[1\] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd edition. 2008.

\[2\] Bruce Dawson. *[Comparing Floating Point Numbers, 2012 Edition](https://randomascii.wordpress.com/2012/02/25/comparing-floating-point-numbers-2012-edition/)*. 2012.

<!-- \[2\] David Goldberg. *[What Every Computer Scientist Should Know About Floating-Point Arithmetic](https://doi.org/10.1145/103162.103163)*. ACM Computing Surveys 23(1), pp. 5–48. 1991. -->
<!-- \[3\] Jean-Michel Muller, Nicolas Brunie, Florent de Dinechin, Claude-Pierre Jeannerod, Mioara Joldes, Vincent Lefèvre, Guillaume Melquiond, Nathalie Revol, and Serge Torres. *Handbook of Floating-Point Arithmetic*, 2nd edition. 2018. -->

\[3\] Jonathan Richard Shewchuck. *[Adaptive Precision Floating-Point Arithmetic and Fast Robust Geometric Predicates](https://www.cs.cmu.edu/~quake/robust.html)*. Discrete & Computational Geometry 18(3), pp. 305–368. 1997.

\[4\] David M. Mount. *[CMSC 754: Computational Geometry (Spring 2020)](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf)*. 2020.

\[5\] Francis S. Hill Jr. *The Pleasures of "Perp Dot" Products*. Graphics Gems IV, pp. 138–148. 1994.

\[6\] Arne Andersson. *Balanced Search Trees Made Simple*. WADS 1993: Algorithms and Data Structures, pp. 60–71. 1993.